# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Panwei Hu
Date: 2025-08-20

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Set up local data directory for this homework
RAW = pathlib.Path('data/raw')
RAW.mkdir(parents=True, exist_ok=True)

print("current working directory:", pathlib.Path.cwd())
# Load environment variables
print(".env file path:", pathlib.Path('.env').absolute())
load_dotenv()
print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))
print('Data directory:', RAW.absolute())

current working directory: /Users/panweihu/Desktop/Desktop_m1/NYU_mfe/bootcamp/camp4/bootcamp_bill_panwei_hu/homework/homework4
.env file path: /Users/panweihu/Desktop/Desktop_m1/NYU_mfe/bootcamp/camp4/bootcamp_bill_panwei_hu/homework/homework4/.env
ALPHAVANTAGE_API_KEY loaded? False
Data directory: /Users/panweihu/Desktop/Desktop_m1/NYU_mfe/bootcamp/camp4/bootcamp_bill_panwei_hu/homework/homework4/data/raw


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

In [3]:
import yfinance as yf 
symbol = 'AAPL'
df = yf.download(symbol, period='2y', interval='1d', progress=False)
df.head()

df.to_csv('AAPL.csv')


/var/folders/dj/t_cw33ws3lb2m3y9lb4jtk640000gn/T/ipykernel_98168/435469504.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period='2y', interval='1d', progress=False)


## Part 1 — API Pull (Required)
Acquire financial time series data for Turtle Trading strategy analysis.
We'll focus on diversified assets: stocks, ETFs, and futures proxies for trend-following analysis.

In [4]:
# FIXED VERSION: Complete data acquisition with corrected yfinance logic
SYMBOLS = ['SPY', 'QQQ', 'IWM', 'EFA', 'EEM',  # Equity ETFs
           'TLT', 'IEF', 'LQD', 'HYG',          # Bond ETFs  
           'GLD', 'SLV', 'USO', 'UNG', 'DBA',   # Commodity ETFs
           'FXE', 'FXY', 'UUP']                  # Currency ETFs

USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
all_data = []

print(f"🚀 Starting data acquisition for {len(SYMBOLS)} symbols...")
print(f"Using {'Alpha Vantage API' if USE_ALPHA else 'yfinance (Yahoo Finance)'}")

for symbol in SYMBOLS:
    print(f"Fetching {symbol}...", end=' ')
    try:
        if USE_ALPHA:
            # Alpha Vantage API logic
            url = 'https://www.alphavantage.co/query'
            params = {
                'function': 'TIME_SERIES_DAILY_ADJUSTED',
                'symbol': symbol,
                'outputsize': 'full',
                'apikey': os.getenv('ALPHAVANTAGE_API_KEY')
            }
            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            js = r.json()
            
            if 'Error Message' in js:
                print(f"❌ API Error: {js['Error Message']}")
                continue
            if 'Note' in js:
                print(f"⚠️  API Limit: {js['Note']}")
                continue
                
            key = [k for k in js if 'Time Series' in k][0]
            df = pd.DataFrame(js[key]).T.reset_index()
            df = df.rename(columns={'index':'date','5. adjusted close':'adj_close'})
            df['date'] = pd.to_datetime(df['date'])
            df['adj_close'] = pd.to_numeric(df['adj_close'])
            df = df[['date','adj_close']].copy()
            
        else:
            # Fixed yfinance logic
            import yfinance as yf
            df = yf.download(symbol, period='2y', interval='1d', progress=False, auto_adjust=True)
            
            if df.empty:
                print("❌ No data")
                continue
                
            df = df.reset_index()
            
            # With auto_adjust=True, the 'Close' column contains adjusted prices
            if 'Close' in df.columns and 'Date' in df.columns:
                df = df[['Date', 'Close']].copy()
                df.columns = ['date', 'adj_close']
            else:
                print(f"❌ Unexpected columns: {list(df.columns)}")
                continue
        
        if df.empty:
            print("❌ Empty data")
            continue
            
        df['symbol'] = symbol
        all_data.append(df)
        print(f"✅ {len(df)} records")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        continue

# Process results
if all_data:
    df_api = pd.concat(all_data, ignore_index=True)
    print(f"\n🎉 SUCCESS! Retrieved {len(df_api):,} total records for {df_api['symbol'].nunique()} symbols")
    
    # Validation
    v_api = validate(df_api, ['date','adj_close','symbol'])
    print(f"📊 Validation: {v_api}")
    
    # Save data
    df_sorted = df_api.sort_values(['symbol', 'date']).reset_index(drop=True)
    saved_path = save_csv(df_sorted, prefix='api', source='alpha' if USE_ALPHA else 'yfinance', 
                         assets='multi', count=len(SYMBOLS))
    print(f"💾 Saved to: {saved_path}")
    
    # Show sample
    print(f"\n📈 Data Summary by Symbol:")
    summary = df_sorted.groupby('symbol').agg({
        'date': ['min', 'max'], 
        'adj_close': ['count', 'mean']
    }).round(2)
    print(summary)
    
else:
    print("❌ No data retrieved! Check your internet connection or API keys.")
    df_api = pd.DataFrame(columns=['date', 'adj_close', 'symbol'])


🚀 Starting data acquisition for 17 symbols...
Using yfinance (Yahoo Finance)
Fetching SPY... ✅ 501 records
Fetching QQQ... ✅ 501 records
Fetching IWM... ✅ 501 records
Fetching EFA... ✅ 501 records
Fetching EEM... ✅ 501 records
Fetching TLT... ✅ 501 records
Fetching IEF... ✅ 501 records
Fetching LQD... ✅ 501 records
Fetching HYG... ✅ 501 records
Fetching GLD... ✅ 501 records
Fetching SLV... ✅ 501 records
Fetching USO... ✅ 501 records
Fetching UNG... ✅ 501 records
Fetching DBA... ✅ 501 records
Fetching FXE... ✅ 501 records
Fetching FXY... ✅ 501 records
Fetching UUP... ✅ 501 records

🎉 SUCCESS! Retrieved 8,517 total records for 17 symbols
📊 Validation: {'missing': [], 'shape': (8517, 3), 'na_total': 0}
Saved data/raw/api_source-yfinance_assets-multi_count-17_20250820-091757.csv
💾 Saved to: data/raw/api_source-yfinance_assets-multi_count-17_20250820-091757.csv

📈 Data Summary by Symbol:
             date            adj_close        
              min        max     count    mean
symbol    

## Part 2 — Scrape a Public Table (Required)
Scrape additional market data relevant to Turtle Trading. We'll get S&P 500 sector information 
and volatility data from public financial websites.

In [7]:
# Try to scrape S&P 500 sector ETF information from a financial website
SCRAPE_URLS = [
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',  # S&P 500 companies
]

headers = {'User-Agent': 'Mozilla/5.0 (Turtle-Trading-Research/1.0)'}
df_scrape = None

for url in SCRAPE_URLS:
    try:
        print(f"Attempting to scrape: {url}")
        resp = requests.get(url, headers=headers, timeout=30)
        resp.raise_for_status()
        
        # Use pandas to read HTML tables directly
        tables = pd.read_html(resp.text)
        
        if 'S%26P_500_companies' in url:
            # Get S&P 500 companies table
            df_scrape = tables[0]  # First table usually contains the companies
            df_scrape = df_scrape[['Symbol', 'Security', 'GICS Sector']].copy()
            df_scrape.columns = ['ticker', 'name', 'sector']
            print(f"  Success: Found {len(df_scrape)} S&P 500 companies")
            break
            
        elif 'Sector_ETFs' in url:
            # Get sector ETF information
            df_scrape = tables[0]  # First table with sector ETFs
            if 'Ticker' in df_scrape.columns:
                df_scrape = df_scrape[['Ticker', 'Name', 'Sector']].copy()
                df_scrape.columns = ['ticker', 'name', 'sector']
            print(f"  Success: Found {len(df_scrape)} sector ETFs")
            break
            
    except Exception as e:
        print(f"  Failed to scrape {url}: {e}")
        continue

# Fallback to demo data if scraping fails
if df_scrape is None or df_scrape.empty:
    print('Scraping failed, using demo sector ETF data')
    demo_data = {
        'ticker': ['XLK', 'XLF', 'XLV', 'XLE', 'XLI', 'XLY', 'XLP', 'XLU', 'XLB', 'XLRE', 'XLC'],
        'name': ['Technology', 'Financials', 'Health Care', 'Energy', 'Industrials', 
                'Consumer Discretionary', 'Consumer Staples', 'Utilities', 'Materials', 
                'Real Estate', 'Communication Services'],
        'sector': ['Technology', 'Financials', 'Health Care', 'Energy', 'Industrials',
                  'Consumer Discretionary', 'Consumer Staples', 'Utilities', 'Materials',
                  'Real Estate', 'Communication Services']
    }
    df_scrape = pd.DataFrame(demo_data)

print(f"\nScraped data shape: {df_scrape.shape}")
print("Sample data:")
print(df_scrape.head())

v_scrape = validate(df_scrape, ['ticker', 'name', 'sector'])
print(f"\nValidation results: {v_scrape}")

Attempting to scrape: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


/var/folders/dj/t_cw33ws3lb2m3y9lb4jtk640000gn/T/ipykernel_98168/1340915299.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


  Success: Found 503 S&P 500 companies

Scraped data shape: (503, 3)
Sample data:
  ticker                 name                  sector
0    MMM                   3M             Industrials
1    AOS          A. O. Smith             Industrials
2    ABT  Abbott Laboratories             Health Care
3   ABBV               AbbVie             Health Care
4    ACN            Accenture  Information Technology

Validation results: {'missing': [], 'shape': (503, 3), 'na_total': 0}


In [8]:
# Save scraped data
if not df_scrape.empty:
    saved_path = save_csv(df_scrape, prefix='scrape', site='wikipedia', table='sp500_sectors')
    print(f"Saved scraped data to {saved_path}")
else:
    print("No scraped data to save!")

Saved data/raw/scrape_site-wikipedia_table-sp500_sectors_20250820-091903.csv
Saved scraped data to data/raw/scrape_site-wikipedia_table-sp500_sectors_20250820-091903.csv


In [26]:
# Additional Data Quality Checks and Summary
print("=== DATA ACQUISITION SUMMARY ===")
print(f"Timestamp: {dt.datetime.now()}")
print(f"Data directory: {RAW.absolute()}")

if not df_api.empty:
    print(f"\n📈 API Data Summary:")
    print(f"  - Total records: {len(df_api):,}")
    print(f"  - Unique symbols: {df_api['symbol'].nunique()}")
    print(f"  - Date range: {df_api['date'].min()} to {df_api['date'].max()}")
    print(f"  - Missing values: {df_api.isnull().sum().sum()}")
    
    # Calculate basic statistics
    latest_prices = df_api.groupby('symbol')['adj_close'].last()
    print(f"\n  Latest prices (sample):")
    for symbol in latest_prices.head(5).index:
        price = latest_prices[symbol]
        print(f"    {symbol}: ${price:.2f}")

if not df_scrape.empty:
    print(f"\n🔍 Scraped Data Summary:")
    print(f"  - Records: {len(df_scrape)}")
    print(f"  - Sectors: {df_scrape['sector'].nunique() if 'sector' in df_scrape.columns else 'N/A'}")
    
print(f"\n💾 Files saved to: {RAW}")
print("✅ Data acquisition complete!")

# Check if turtle_project data directory was created
if RAW.exists():
    files = list(RAW.glob("*.csv"))
    print(f"\n📁 Raw data files ({len(files)}):")
    for f in files:
        size_mb = f.stat().st_size / (1024*1024)
        print(f"  - {f.name} ({size_mb:.1f} MB)")
else:
    print("❌ Data directory not found!")


=== DATA ACQUISITION SUMMARY ===
Timestamp: 2025-08-17 21:17:20.678184
Data directory: /Users/panweihu/Desktop/Desktop_m1/NYU_mfe/bootcamp/camp4/bootcamp_bill_panwei_hu/homework/../turtle_project/data/raw

📈 API Data Summary:
  - Total records: 8,534
  - Unique symbols: 17
  - Date range: 2023-08-16 00:00:00 to 2025-08-15 00:00:00
  - Missing values: 0

  Latest prices (sample):
    DBA: $27.06
    EEM: $49.94
    EFA: $92.19
    FXE: $107.96
    FXY: $62.54

🔍 Scraped Data Summary:
  - Records: 503
  - Sectors: 11

💾 Files saved to: ../turtle_project/data/raw
✅ Data acquisition complete!

📁 Raw data files (5):
  - api_source-yfinance_assets-multi_count-17_20250817-211709.csv (0.3 MB)
  - api_source-yfinance_assets-multi_count-17_20250817-211655.csv (0.3 MB)
  - scrape_site-wikipedia_table-sp500_sectors_20250817-205825.csv (0.0 MB)
  - scrape_site-wikipedia_table-sp500_sectors_20250817-211718.csv (0.0 MB)
  - api_fixed_source-yfinance_assets-multi_count-17_20250817-211626.csv (0.3 MB)


## Documentation

### Data Sources for Turtle Trading Project

#### API Sources:
- **Primary**: Alpha Vantage API (https://www.alphavantage.co/query)
  - Endpoint: TIME_SERIES_DAILY_ADJUSTED
  - Parameters: symbol, outputsize=full, apikey
  - Fallback: yfinance library for Yahoo Finance data
  - Assets: 17 diversified ETFs covering equities, bonds, commodities, currencies
  - Time period: Up to 2 years of daily data

#### Web Scraping Sources:
- **S&P 500 Companies**: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
- **Sector ETFs**: https://en.wikipedia.org/wiki/SPDR_Select_Sector_ETFs
- **Method**: pandas.read_html() for structured table extraction
- **Fallback**: Hardcoded sector ETF data if scraping fails

### Asset Universe for Turtle Trading:
- **Equity ETFs**: SPY, QQQ, IWM, EFA, EEM (US large/small cap, international)
- **Bond ETFs**: TLT, IEF, LQD, HYG (treasury, corporate, high yield)
- **Commodity ETFs**: GLD, SLV, USO, UNG, DBA (metals, energy, agriculture)
- **Currency ETFs**: FXE, FXY, UUP (euro, yen, dollar strength)

### Assumptions & Risks:

#### Rate Limits & API Constraints:
- Alpha Vantage: 5 calls/minute, 500 calls/day (free tier)
- Wikipedia: No formal rate limits, but respectful scraping with delays
- yfinance: Unofficial API, subject to Yahoo Finance changes

#### Data Quality Risks:
- **Schema changes**: Wikipedia table structures may change
- **Selector fragility**: HTML parsing depends on consistent table structure  
- **Missing data**: Some ETFs may have limited history or gaps
- **Corporate actions**: Splits/dividends handled differently across sources

#### Operational Risks:
- **API key management**: Stored in .env file (excluded from git)
- **Network failures**: Timeout handling and retry logic needed
- **Data staleness**: Daily data may have 1-day lag
- **Cost sensitivity**: Slippage assumptions critical for Turtle strategy viability

### Validation & Quality Control:
- **Required columns**: date, adj_close, symbol for API data
- **Data completeness**: Check for missing values and date gaps
- **Date range validation**: Ensure sufficient history for backtesting
- **Price reasonableness**: Sanity checks on price levels and returns

### Environment Setup:
- ✅ `.env` file excluded from git via .gitignore
- ✅ Data saved to turtle_project/data/raw/ directory
- ✅ Fallback mechanisms for both API and scraping failures